# Lab 6: Build a Streamlit Frontend

In this lab, you'll create a customer-facing **Streamlit web application** that connects to the deployed AgentCore Runtime.

## What you'll learn
- How to build a chat UI with Streamlit
- How to integrate Cognito authentication
- How to stream responses from AgentCore Runtime
- How to manage sessions and conversation history

## Architecture

<div style="text-align:left">
    <img src="images/architecture_lab6_streamlit.png" width="100%"/>
</div>

## Step 1: Install Frontend Dependencies

In [ ]:
%pip install -q -r frontend/requirements.txt

## Step 2: Review Frontend Architecture

The frontend consists of:

| File | Purpose |
|------|--------|
| `frontend/main.py` | Streamlit app entry point with Cognito auth |
| `frontend/chat.py` | ChatManager class for agent invocation and streaming |
| `frontend/chat_utils.py` | URL formatting, SSM helpers, safe markdown |

In [ ]:
# Verify prerequisites are in place
import os
import json
import sys

os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

from utils.aws_helpers import get_ssm_parameter, get_customer_support_secret

# Check runtime is deployed
try:
    runtime_arn = get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    print(f"Runtime ARN: {runtime_arn}")
except Exception:
    print("ERROR: Runtime not deployed. Complete Lab 4 first.")

# Check secrets are available
try:
    secret = get_customer_support_secret()
    config = json.loads(secret)
    print(f"Cognito Pool: {config['pool_id']}")
    print(f"Client ID: {config['client_id']}")
except Exception:
    print("ERROR: Cognito not configured. Complete Lab 3 first.")

## Step 3: Launch Streamlit Application

In [ ]:
# Get the Streamlit URL
try:
    with open("/opt/ml/metadata/resource-metadata.json", "r") as f:
        metadata = json.load(f)
    domain_id = metadata["DomainId"]
    space_name = metadata["SpaceName"]
    
    import boto3
    sagemaker_client = boto3.client("sagemaker")
    response = sagemaker_client.describe_space(DomainId=domain_id, SpaceName=space_name)
    streamlit_url = response["Url"] + "/proxy/8501/"
except Exception:
    streamlit_url = "http://localhost:8501"

print(f"Streamlit URL: {streamlit_url}")
print("\nLogin credentials:")
print("  Username: testuser")
print("  Password: MyPassword123!")

In [ ]:
import subprocess

# Launch Streamlit as a background process
process = subprocess.Popen(
    ["streamlit", "run", "main.py", "--server.port", "8501", "--server.address", "0.0.0.0"],
    cwd="frontend",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

print(f"Streamlit launched (PID: {process.pid})")
print(f"Open: {streamlit_url}")
print("\nTo stop: Run process.terminate() in a new cell")

## Step 4: Test the Application

Open the Streamlit URL in your browser and test with these scenarios:

1. **Product inquiry**: "Tell me about your laptop specifications"
2. **Return policy**: "What's the return policy for smartphones?"
3. **Technical support**: "My headphones won't pair via Bluetooth"
4. **Warranty check**: "Can you check warranty for serial number ABC12345?"
5. **Memory test**: Ask a follow-up question referencing earlier context

### What to observe:
- Cognito login flow
- Real-time streaming responses
- Response timing metrics
- Session continuity across messages

<div style="text-align:left">
    <img src="images/lab5_streamlit_login.png"/>
</div>
<div>
    <img src="images/lab5_welcome_user.png"/>
</div>

<div style="text-align:left">
    <img src="images/lab5_agent_question.png" width="75%"/>
</div>

<div style="text-align:left">
    <img src="images/lab5_agent_chat_history.png" width="75%"/>
</div>


## Summary

In this lab, you built and launched a Streamlit frontend:

1. **Cognito authentication** for secure access
2. **Chat interface** with real-time streaming
3. **Session management** for conversation continuity
4. **Response formatting** with timing metrics

In **Lab 7**, we'll clean up all AWS resources.